# KNN (K-Nearest-Neighbors)

KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:

In [1]:
import pandas as pd

# Realizando a leitura do arquivo de avaliações dos filmes a serem analisados
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))

# Exibindo as primeiras linhas do arquivo de avaliações em um DataFrame
ratings.head()


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:

In [2]:
import numpy as np

# Agrupando as avaliações por filme e calculando a média e a quantidade de avaliações
movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

C:\Users\cainh\AppData\Local\Temp\ipykernel_2572\2628946610.py:3: FutureWarning: The provided callable <function mean at 0x0000026115B8DBC0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [13]:
# Normalizando a quantidade de avaliações por filme, utilizando a fórmula de normalização min-max
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])

# Aplicando a normalização min-max
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

# Exibindo as primeiras linhas do DataFrame com a quantidade de avaliações normalizadas
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [4]:
movieDict = {}

# abrindo o arquivo referente aos filmes e armazenando as informações em um dicionário para posterior manipulação

with open(r'ml-100k/u.item', encoding="ISO-8859-1") as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


For example, here's the record we end up with for movie ID 1, "Toy Story":

In [5]:
print(movieDict[1]) # printando as informações do filme com ID 1

('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.7735849056603774, 3.8783185840707963)


Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [ ]:
from scipy import spatial

# Função para calcular a "distância" entre dois filmes
def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB) # calculando a distância entre os gêneros dos filmes utilizando a distância cosseno
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB) # calculando a distância entre a popularidade dos filmes
    return genreDistance + popularityDistance
    
# Calculando a distância entre os filmes com ID 2 e 4
ComputeDistance(movieDict[2], movieDict[4])



0.8004574042309892

Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:

In [7]:
print(movieDict[2]) # printando as informações do filme com ID 2
print(movieDict[4]) # printando as informações do filme com ID 4


('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)


Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. When the sort those by distance, and print out the K nearest neighbors:

In [8]:
import operator

# Função para obter os K vizinhos mais próximos de um filme

def getNeighbors(movieID, K):
    distances = []
    
    # Percorrendo todos os filmes
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie]) # definindo a distância entre os filmes
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1)) # ordenando as distâncias
    neighbors = [] # armazenando os K vizinhos mais próximos

    # Adicionando os K vizinhos mais próximos em neighbors
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10 # definindo o número de vizinhos mais próximos, o k.
avgRating = 0 # inicializando a variável para armazenar a média das avaliações dos vizinhos
neighbors = getNeighbors(1, K) # utilizando a função getNeighbors para obter os K vizinhos mais próximos do filme 1

# Exibindo os K vizinhos mais próximos do filme 1 e a média das avaliações desses vizinhos
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    
avgRating /= K

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:

In [9]:
avgRating

3.3445905900235564

How does this compare to Toy Story's actual average rating?

In [10]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

Not too bad!


## Activity

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?

### Resolvendo o exercício proposto e aplicando meus conhecimentos aprendidos em aula:

Utilizando o metodo de distância euclidiana 

In [44]:
# Função para definir a distância entre dois filmes, utilizando a distância euclidiana
def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.euclidean(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    

K = 10
avgRating = 0
neighbors = getNeighbors(1, K)

# Imprime os 10 filmes mais próximos ao filme 1
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]) +  "    " + str(movieDict[neighbor][1]))
    
avgRating /= K

Aladdin and the King of Thieves (1996) 2.8461538461538463    [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Aladdin (1992) 3.8127853881278537    [0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0]
Liar Liar (1997) 3.156701030927835    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
George of the Jungle (1997) 2.685185185185185    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Beavis and Butt-head Do America (1996) 2.7884615384615383    [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Home Alone (1990) 3.0875912408759123    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Jungle2Jungle (1997) 2.4393939393939394    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Wrong Trousers, The (1993) 4.466101694915254    [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
101 Dalmatians (1996) 2.908256880733945    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Pinocchio (1940) 3.6732673267326734    [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


É possível observar que a distância entre a média de avaliações e a avaliação de toy story foi aumentada, ou seja, esse metodo teve um resultado pior que o metodo cosseno, nesse contexto.

In [37]:
avgRating

3.186389807150798

Utilizando o metodo de distância de hamming

In [41]:
# Função para definir a distância entre dois filmes, utilizando a distância de correlação
def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.correlation(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance

K = 12
avgRating = 0
neighbors = getNeighbors(1, K)
print(movieDict[1][1])
# Imprime os 12 filmes mais próximos ao filme 1
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]) + "    " + str(movieDict[neighbor][1]))
    
avgRating /= K
    

[0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Liar Liar (1997) 3.156701030927835    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Aladdin (1992) 3.8127853881278537    [0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0]
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975    [0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Monty Python and the Holy Grail (1974) 4.0664556962025316    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Full Monty, The (1997) 3.926984126984127    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
George of the Jungle (1997) 2.685185185185185    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Beavis and Butt-head Do America (1996) 2.7884615384615383    [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Birdcage, The (1996) 3.4436860068259385    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Aladdin and the King of Thieves (1996) 2.8461538461538463    [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Home Alone (1990) 3.0875912408759123    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Jungle2Jungle (1997) 2.4393939393939394    [0 0 0 0 1 1 0 0

É possível observar que a distância entre a média de avaliações e a avaliação de toy story foi diminuída, ou seja, esse metodo teve um resultado melhor que o metodo cosseno, nesse contexto.

In [35]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

In [42]:
avgRating

3.362616794545396